# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data = pd.read_csv("../output_data/cities.csv")
clean_city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Coyhaique,-45.5752,-72.0662,74.01,30,0,5.75,CL,1643142441
1,Port Alfred,-33.5906,26.8910,72.25,83,67,8.37,ZA,1643142442
2,Punta Arenas,-53.1500,-70.9167,51.91,50,75,29.93,CL,1643142355
3,Chui,-33.6971,-53.4616,83.03,70,93,10.49,UY,1643142444
4,Pochutla,15.7432,-96.4661,87.82,39,0,11.30,MX,1643142446


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = clean_city_data[["Lat", "Lng"]].astype(float)

humidity = clean_city_data["Humidity"].astype(float)

In [4]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow cities list based on ideal weather conditions
ideal_cities = clean_city_data.loc[((clean_city_data["Max Temp"] > 65) & 
                                   (clean_city_data["Max Temp"] < 80) & 
                                   (clean_city_data["Wind Speed"] < 10) & 
                                   (clean_city_data["Cloudiness"] == 0)), :]

ideal_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Coyhaique,-45.5752,-72.0662,74.01,30,0,5.75,CL,1643142441
15,Kapaa,22.0752,-159.3190,75.52,66,0,8.05,US,1643142460
32,Cape Town,-33.9258,18.4232,68.41,73,0,7.00,ZA,1643142486
43,Ancud,-41.8697,-73.8203,73.22,65,0,7.76,CL,1643142500


In [6]:
#Drop rows with null values
hotel_df = ideal_cities.dropna(how = "any")

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Coyhaique,-45.5752,-72.0662,74.01,30,0,5.75,CL,1643142441
15,Kapaa,22.0752,-159.3190,75.52,66,0,8.05,US,1643142460
32,Cape Town,-33.9258,18.4232,68.41,73,0,7.00,ZA,1643142486
43,Ancud,-41.8697,-73.8203,73.22,65,0,7.76,CL,1643142500


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Create column in data frame for hotel name
hotel_df["Hotel Name"] = ""

#Set parameters for hotel search
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key
}

#Loop through data frame to call
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    # Change location each iteration of loop
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and location coordinates
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and convert to JSON
    name_address = requests.get(base_url, params=params).json()
    
    #Exception handling for any missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("No data available. Skipping location")


hotel_df  

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Coyhaique,-45.5752,-72.0662,74.01,30,0,5.75,CL,1643142441,Nomades Hotel Boutique
15,Kapaa,22.0752,-159.3190,75.52,66,0,8.05,US,1643142460,Hotel Coral Reef
32,Cape Town,-33.9258,18.4232,68.41,73,0,7.00,ZA,1643142486,"Radisson Blu Hotel Waterfront, Cape Town"
43,Ancud,-41.8697,-73.8203,73.22,65,0,7.76,CL,1643142500,Panamericana Hotel Ancud


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# Create hotel symbol layer using above template info
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content = hotel_info
)

#Create map and add layers
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))